In [1]:
from magma import *
from magma.primitives import DefineRegister

N = 4
Register4 = DefineRegister(4, has_reset=True)
T = Bits(N)

class ResetShiftRegister(Circuit):
    name = "ResetShiftRegister"
    IO = ["I", In(T), "reset", In(Bit), "O", Out(T)] + ClockInterface()
    @classmethod
    def definition(io):
        regs = [Register4().reset(io.reset) for _ in range(4)]
        wireclock(io, regs)
        wire(io.I, getattr(regs[0], "in"))
        fold(regs, foldargs={"in":"out"})
        wire(regs[-1].out, io.O)

In [2]:
from magma.backend.verilog import compile as compile_verilog
print(compile_verilog(ResetShiftRegister))

compiling ResetShiftRegister
module ResetShiftRegister (input [3:0] I, input  reset, output [3:0] O, input  CLK);
wire [3:0] inst0_out;
wire [3:0] inst1_out;
wire [3:0] inst2_out;
wire [3:0] inst3_out;
coreir_reg_PR #(.width(4)) inst0 (.in(I), .clk(CLK), .out(inst0_out), .rst(reset));
coreir_reg_PR #(.width(4)) inst1 (.in(inst0_out), .clk(CLK), .out(inst1_out), .rst(reset));
coreir_reg_PR #(.width(4)) inst2 (.in(inst1_out), .clk(CLK), .out(inst2_out), .rst(reset));
coreir_reg_PR #(.width(4)) inst3 (.in(inst2_out), .clk(CLK), .out(inst3_out), .rst(reset));
assign O = inst3_out;
endmodule




In [3]:
from magma.simulator import PythonSimulator
from magma.bit_vector import BitVector

simulator = PythonSimulator(ResetShiftRegister, clock=ResetShiftRegister.CLK)
outputs = []
simulator.set_value(ResetShiftRegister.reset, True)
for i in range(0, 1 << N):
    simulator.set_value(ResetShiftRegister.I, BitVector(i, N))
    
    for j in range(2):
        if i == 9:
            if j == 0:
                simulator.set_value(ResetShiftRegister.reset, False)
            else:
                simulator.set_value(ResetShiftRegister.reset, True)
        simulator.step()
        simulator.evaluate()
        O = simulator.get_value(ResetShiftRegister.O)
        outputs.append(BitVector(O).as_int())

print(outputs)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 11, 11, 12, 12]
